In [ ]:
# ============================================================================
# CELDA 1: INSTALACIÓN DE LIBRERÍAS NECESARIAS
# ============================================================================

import sys
import subprocess

print("="*70)
print("VERIFICANDO LIBRERÍAS NECESARIAS")
print("="*70)
print()

librerias = {
    'pandas': 'Manipulación y análisis de datos',
    'numpy': 'Operaciones numéricas',
    'openpyxl': 'Leer/escribir archivos Excel',
    'matplotlib': 'Gráficos y visualizaciones',
    'seaborn': 'Visualizaciones estadísticas',
    'plotly': 'Gráficos interactivos',
    'scipy': 'Funciones científicas',
    'statsmodels': 'Series temporales y predicción',
    'scikit-learn': 'Machine Learning'
}

for libreria, descripcion in librerias.items():
    try:
        __import__(libreria)
        print(f"   ✅ {libreria:.<20} {descripcion} - YA INSTALADA")
    except ImportError:
        print(f"   ❌ {libreria:.<20} {descripcion} - FALTANTE")
        try:
            print(f"      Instalando {libreria}...")
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", libreria],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL
            )
            print(f"      ✅ {libreria} instalada correctamente")
        except:
            print(f"      ⚠️  Error instalando {libreria}")

print()
print("="*70)
print("VERIFICANDO VERSIONES")
print("="*70)

import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns

print(f"\n✅ Python:      {sys.version.split()[0]}")
print(f"✅ Pandas:      {pd.__version__}")
print(f"✅ NumPy:       {np.__version__}")
print(f"✅ Matplotlib:  {matplotlib.__version__}")
print(f"✅ Seaborn:     {sns.__version__}")

print()
print("="*70)
print("✅ SISTEMA LISTO")
print("="*70)
print()

In [ ]:
# ============================================================================
# CELDA 2: CONFIGURACIÓN DEL ANÁLISIS
# ============================================================================

import warnings
warnings.filterwarnings('ignore')

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 100)

# Configuración de matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 11

print("="*70)
print("CONSTANTES DEL SISTEMA")
print("="*70)
print()

# ⚙️ CONSTANTES DEL SISTEMA (EDITABLES)
TOTAL_BINS = 30000          # Total de bins en el sistema (constante)
UMBRAL_MINIMO = 5000        # Mínimo de bins que debe haber en planta
UMBRAL_AMARILLO = 8000      # Umbral de advertencia
DIAS_PROYECCION = 7         # Días hacia adelante para proyección

print(f"🏭 Total bins en el sistema:     {TOTAL_BINS:>10,}")
print(f"🔴 Umbral crítico (mínimo):      {UMBRAL_MINIMO:>10,}")
print(f"🟡 Umbral de advertencia:        {UMBRAL_AMARILLO:>10,}")
print(f"📅 Días para proyección:         {DIAS_PROYECCION:>10}")
print()
print("="*70)
print("✅ CONFIGURACIÓN COMPLETADA")
print("="*70)
print()

In [ ]:
# ============================================================================
# CELDA 3: CARGAR ARCHIVO MOVIMIENTOS.xlsx
# ============================================================================

print("="*70)
print("CARGANDO ARCHIVO: MOVIMIENTOS.xlsx")
print("="*70)
print()

try:
    # Cargar archivo - RUTA MODIFICADA PARA ANACONDA
    ruta_archivo = r"C:\JUGOS\resultados\MOVIMIENTOS.xlsx"
    MOVIMIENTOS = pd.read_excel(ruta_archivo)

    print(f"✅ Archivo cargado exitosamente desde: {ruta_archivo}")
    print()
    print(f"📊 Total de registros: {len(MOVIMIENTOS):,}")
    print(f"📋 Columnas: {', '.join(MOVIMIENTOS.columns)}")
    print()

    # Validar columnas requeridas
    columnas_requeridas = ['FECHA', 'PROVEEDOR', 'NOMBRE', 'ENVASE', 'CANTIDAD', 'MOVIMIENTO']
    columnas_faltantes = [col for col in columnas_requeridas if col not in MOVIMIENTOS.columns]

    if columnas_faltantes:
        print(f"⚠️  COLUMNAS FALTANTES: {', '.join(columnas_faltantes)}")
        raise ValueError("Faltan columnas requeridas")

    print("✅ Todas las columnas requeridas están presentes")
    print()

    # Validar valores en MOVIMIENTO
    valores_movimiento = MOVIMIENTOS['MOVIMIENTO'].unique()
    print(f"📝 Tipos de movimiento: {', '.join(valores_movimiento)}")

    if not all(x in ['ENTRADA', 'SALIDA'] for x in valores_movimiento):
        print(f"⚠️  ADVERTENCIA: Valores inesperados en MOVIMIENTO")

    print()
    print("="*70)
    print("INFORMACIÓN GENERAL")
    print("="*70)
    print()
    print(MOVIMIENTOS.info())
    print()
    print("="*70)
    print("PRIMEROS REGISTROS")
    print("="*70)
    print()
    print(MOVIMIENTOS.head(10))

except FileNotFoundError:
    print(f"❌ ERROR: No se encontró el archivo en la ruta: {ruta_archivo}")
    print()
    print("📝 INSTRUCCIONES PARA ANACONDA:")
    print("   1. Verifica que la ruta del archivo sea correcta")
    print("   2. Asegúrate de que el archivo MOVIMIENTOS.xlsx existe en esa ubicación")
    print("   3. Si el archivo está en otra ubicación, modifica la variable 'ruta_archivo'")
    print()
    raise

except Exception as e:
    print(f"❌ ERROR: {str(e)}")
    raise

In [ ]:
# ============================================================================
# CELDA 4: CALCULAR BALANCE DIARIO Y STOCK (SISTEMA CERRADO)
# ============================================================================

print("="*70)
print("ANÁLISIS DE BALANCE DIARIO - SISTEMA CERRADO")
print("="*70)
print()

print(f"🏭 Total de bins en el sistema: {TOTAL_BINS:,}")
print(f"⚠️  Umbral mínimo en planta: {UMBRAL_MINIMO:,}")
print(f"🟡 Umbral de advertencia: {UMBRAL_AMARILLO:,}")
print()

# Crear columna de FECHA_DIA (solo fecha, sin hora)
MOVIMIENTOS['FECHA_DIA'] = pd.to_datetime(MOVIMIENTOS['FECHA']).dt.date
MOVIMIENTOS['FECHA_DIA'] = pd.to_datetime(MOVIMIENTOS['FECHA_DIA'])

# Agrupar por día y tipo de movimiento
balance_diario = MOVIMIENTOS.groupby(['FECHA_DIA', 'MOVIMIENTO'])['CANTIDAD'].sum().unstack(fill_value=0)
balance_diario = balance_diario.reset_index()

# Si no existe alguna columna, crearla con 0
if 'ENTRADA' not in balance_diario.columns:
    balance_diario['ENTRADA'] = 0
if 'SALIDA' not in balance_diario.columns:
    balance_diario['SALIDA'] = 0

# Ordenar por fecha
balance_diario = balance_diario.sort_values('FECHA_DIA').reset_index(drop=True)

# Calcular flujo neto diario
balance_diario['NETO_DIARIO'] = balance_diario['ENTRADA'] - balance_diario['SALIDA']

# Calcular acumulados
balance_diario['ENTRADAS_ACUM'] = balance_diario['ENTRADA'].cumsum()
balance_diario['SALIDAS_ACUM'] = balance_diario['SALIDA'].cumsum()

# 🔥 FÓRMULAS CLAVE DEL SISTEMA CERRADO:
balance_diario['BINS_EN_PLANTA'] = TOTAL_BINS + balance_diario['ENTRADAS_ACUM'] - balance_diario['SALIDAS_ACUM']
balance_diario['BINS_EN_CAMPOS'] = balance_diario['SALIDAS_ACUM'] - balance_diario['ENTRADAS_ACUM']

# Verificación (debe dar siempre 30,000)
balance_diario['VERIFICACION'] = balance_diario['BINS_EN_PLANTA'] + balance_diario['BINS_EN_CAMPOS']

# Identificar niveles de alerta
balance_diario['NIVEL_ALERTA'] = 'NORMAL'
balance_diario.loc[balance_diario['BINS_EN_PLANTA'] < UMBRAL_AMARILLO, 'NIVEL_ALERTA'] = 'ADVERTENCIA'
balance_diario.loc[balance_diario['BINS_EN_PLANTA'] < UMBRAL_MINIMO, 'NIVEL_ALERTA'] = 'CRITICO'

print("✅ Balance diario calculado")
print()
print(f"📅 Período analizado: {balance_diario['FECHA_DIA'].min().strftime('%d/%m/%Y')} a {balance_diario['FECHA_DIA'].max().strftime('%d/%m/%Y')}")
print(f"📊 Total días con movimientos: {len(balance_diario):,}")
print()

# Estadísticas generales
print("="*70)
print("ESTADÍSTICAS GENERALES")
print("="*70)
print()
print(f"📥 Total ENTRADAS:  {balance_diario['ENTRADA'].sum():>15,} bins")
print(f"📤 Total SALIDAS:   {balance_diario['SALIDA'].sum():>15,} bins")
print(f"📊 NETO TOTAL:      {balance_diario['NETO_DIARIO'].sum():>15,} bins")
print()
print(f"🏭 Stock MÍNIMO en planta:      {balance_diario['BINS_EN_PLANTA'].min():>10,} bins")
print(f"🏭 Stock MÁXIMO en planta:      {balance_diario['BINS_EN_PLANTA'].max():>10,} bins")
print(f"🏭 Stock PROMEDIO en planta:    {balance_diario['BINS_EN_PLANTA'].mean():>10,.0f} bins")
print(f"🏭 Stock ACTUAL en planta:      {balance_diario['BINS_EN_PLANTA'].iloc[-1]:>10,} bins")
print()
print(f"📦 Máximo bins en CAMPOS:       {balance_diario['BINS_EN_CAMPOS'].max():>10,} bins")
print(f"📦 Bins ACTUALES en CAMPOS:     {balance_diario['BINS_EN_CAMPOS'].iloc[-1]:>10,} bins")
print()

# Análisis por nivel de alerta
print("="*70)
print("ANÁLISIS DE ALERTAS")
print("="*70)
print()

dias_normales = len(balance_diario[balance_diario['NIVEL_ALERTA'] == 'NORMAL'])
dias_advertencia = len(balance_diario[balance_diario['NIVEL_ALERTA'] == 'ADVERTENCIA'])
dias_criticos = len(balance_diario[balance_diario['NIVEL_ALERTA'] == 'CRITICO'])

total_dias = len(balance_diario)

print(f"🟢 Días NORMALES (> {UMBRAL_AMARILLO:,}):     {dias_normales:>5} días ({dias_normales/total_dias*100:>5.1f}%)")
print(f"🟡 Días ADVERTENCIA ({UMBRAL_MINIMO:,}-{UMBRAL_AMARILLO:,}): {dias_advertencia:>5} días ({dias_advertencia/total_dias*100:>5.1f}%)")
print(f"🔴 Días CRÍTICOS (< {UMBRAL_MINIMO:,}):        {dias_criticos:>5} días ({dias_criticos/total_dias*100:>5.1f}%)")
print()

# Mostrar días críticos
if dias_criticos > 0:
    print("="*70)
    print(f"⚠️  DÍAS CRÍTICOS DETECTADOS: {dias_criticos}")
    print("="*70)
    print()
    dias_criticos_df = balance_diario[balance_diario['NIVEL_ALERTA'] == 'CRITICO'].copy()
    dias_criticos_df = dias_criticos_df.nsmallest(10, 'BINS_EN_PLANTA')
    print("Top 10 días más críticos:")
    print()
    print(dias_criticos_df[['FECHA_DIA', 'BINS_EN_PLANTA', 'BINS_EN_CAMPOS', 'ENTRADA', 'SALIDA']].to_string(index=False))
else:
    print(f"✅ No hay días críticos (stock siempre > {UMBRAL_MINIMO:,})")

print()
print("="*70)
print("PRIMEROS 10 DÍAS DEL BALANCE")
print("="*70)
print()
print(balance_diario[['FECHA_DIA', 'ENTRADA', 'SALIDA', 'NETO_DIARIO',
                       'BINS_EN_PLANTA', 'BINS_EN_CAMPOS', 'NIVEL_ALERTA']].head(10).to_string(index=False))
print()
print("="*70)
print("✅ CELDA 4 COMPLETADA")
print("="*70)

In [ ]:
# ============================================================================
# CELDA 5: GRÁFICO DE ÁREAS APILADAS - DISTRIBUCIÓN PLANTA VS CAMPOS
# ============================================================================

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

print("Generando gráfico de distribución...")

# Crear figura
fig, ax = plt.subplots(figsize=(18, 9))

# ============================================================================
# ÁREAS: Verde (planta) + Roja (campos) = 30,000 bins (sistema cerrado)
# ============================================================================

# Área VERDE: Bins disponibles en planta (de 0 hasta BINS_EN_PLANTA)
ax.fill_between(balance_diario['FECHA_DIA'],
                0,
                balance_diario['BINS_EN_PLANTA'],
                color='#2ECC71', alpha=0.7, label='Bins en PLANTA (disponibles)')

# Área ROJA: Bins comprometidos en campos (de BINS_EN_PLANTA hasta 30,000)
ax.fill_between(balance_diario['FECHA_DIA'],
                balance_diario['BINS_EN_PLANTA'],
                TOTAL_BINS,
                color='#E74C3C', alpha=0.7, label='Bins en CAMPOS (comprometidos)')

# ============================================================================
# LÍNEAS DE REFERENCIA
# ============================================================================

# Línea negra: Total del sistema (30,000)
ax.axhline(y=TOTAL_BINS, color='black', linestyle='-', linewidth=2,
           alpha=0.5, label=f'Total sistema: {TOTAL_BINS:,} bins', zorder=5)

# Línea naranja: Umbral de advertencia
ax.axhline(y=UMBRAL_AMARILLO, color='orange', linestyle='--', linewidth=2,
           label=f'Advertencia: {UMBRAL_AMARILLO:,} bins', alpha=0.8, zorder=5)

# Línea roja: Umbral crítico
ax.axhline(y=UMBRAL_MINIMO, color='darkred', linestyle='--', linewidth=3,
           label=f'CRÍTICO: {UMBRAL_MINIMO:,} bins', alpha=0.9, zorder=5)

# ============================================================================
# MARCADORES: Días críticos
# ============================================================================

dias_criticos = balance_diario[balance_diario['NIVEL_ALERTA'] == 'CRITICO']
if len(dias_criticos) > 0:
    ax.scatter(dias_criticos['FECHA_DIA'], dias_criticos['BINS_EN_PLANTA'],
               color='darkred', s=100, marker='X', linewidths=2,
               edgecolors='white', zorder=10, label='Días CRÍTICOS')

# ============================================================================
# FORMATO DEL GRÁFICO
# ============================================================================

# Títulos
ax.set_title('Distribución de Bins: Planta vs Campos (Sistema Cerrado)\nJugos S.A.',
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Fecha', fontsize=14, fontweight='bold')
ax.set_ylabel('Cantidad de Bins', fontsize=14, fontweight='bold')

# Eje Y: formato con separadores de miles
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x):,}'))
ax.set_ylim(0, TOTAL_BINS + 2000)

# Eje X: fechas cada 2 semanas
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m/%Y'))
ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=2))
plt.xticks(rotation=45, ha='right')

# Grid y leyenda
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
ax.legend(loc='upper right', fontsize=11, framealpha=0.95, shadow=True)

plt.tight_layout()
plt.show()

print("✅ Gráfico generado correctamente")

In [ ]:
# ============================================================================
# CELDA 6: ANÁLISIS DE ALERTA TEMPRANA (SOLO TEXTO)
# ============================================================================

import numpy as np
from datetime import timedelta

print("\n" + "="*80)
print("                    SISTEMA DE ALERTA TEMPRANA - JUGOS S.A.")
print("="*80)
print()

# ============================================================================
# CALCULAR TENDENCIA (Regresión lineal últimos 30 días)
# ============================================================================

dias_para_tendencia = min(30, len(balance_diario))
datos_recientes = balance_diario.tail(dias_para_tendencia).copy()

# Convertir fechas a números para regresión
datos_recientes['dias_num'] = (datos_recientes['FECHA_DIA'] - datos_recientes['FECHA_DIA'].min()).dt.days

# Regresión lineal
x = datos_recientes['dias_num'].values
y = datos_recientes['BINS_EN_PLANTA'].values
coef = np.polyfit(x, y, 1)  # [pendiente, intersección]
pendiente = coef[0]

# Crear proyección a 7 días
fecha_inicio = balance_diario['FECHA_DIA'].iloc[-1]
fechas_proyeccion = [fecha_inicio + timedelta(days=i) for i in range(1, 8)]
dias_proyeccion = np.array(range(x[-1] + 1, x[-1] + 8))
valores_proyeccion = np.polyval(coef, dias_proyeccion)

# ============================================================================
# DATOS ACTUALES
# ============================================================================

stock_actual = balance_diario['BINS_EN_PLANTA'].iloc[-1]
fecha_actual = balance_diario['FECHA_DIA'].iloc[-1]
stock_proyectado_7dias = valores_proyeccion[-1]

print("📊 ESTADO ACTUAL DEL STOCK")
print("-" * 80)
print(f"   Fecha:                        {fecha_actual.strftime('%d/%m/%Y')}")
print(f"   Stock en planta:              {stock_actual:>15,.0f} bins")
print(f"   Stock proyectado (7 días):    {stock_proyectado_7dias:>15,.0f} bins")
print()

# ============================================================================
# ANÁLISIS DE TENDENCIA
# ============================================================================

print("📈 ANÁLISIS DE TENDENCIA (últimos 30 días)")
print("-" * 80)
print(f"   Cambio promedio:              {pendiente:>15,.1f} bins/día")

if pendiente < 0:
    print(f"   Dirección:                    {'BAJANDO':>20} ⬇️")
    velocidad = "RÁPIDA" if abs(pendiente) > 200 else "MODERADA" if abs(pendiente) > 100 else "LENTA"
    print(f"   Velocidad:                    {velocidad:>20}")
elif pendiente > 0:
    print(f"   Dirección:                    {'SUBIENDO':>20} ⬆️")
    print(f"   Estado:                       {'RECUPERACIÓN':>20}")
else:
    print(f"   Dirección:                    {'ESTABLE':>20} ➡️")

print()

# ============================================================================
# CALCULAR DÍAS HASTA UMBRALES
# ============================================================================

dias_hasta_critico = None
dias_hasta_amarillo = None
fecha_critico = None
fecha_amarillo = None

if pendiente < 0:
    # Calcular días hasta crítico
    if stock_actual > UMBRAL_MINIMO:
        dias_hasta_critico = (stock_actual - UMBRAL_MINIMO) / abs(pendiente)
        fecha_critico = fecha_actual + timedelta(days=int(dias_hasta_critico))
    
    # Calcular días hasta amarillo
    if stock_actual > UMBRAL_AMARILLO:
        dias_hasta_amarillo = (stock_actual - UMBRAL_AMARILLO) / abs(pendiente)
        fecha_amarillo = fecha_actual + timedelta(days=int(dias_hasta_amarillo))

# ============================================================================
# GENERAR ALERTA SEGÚN NIVEL
# ============================================================================

print("🚨 EVALUACIÓN DE RIESGO")
print("=" * 80)
print()

# Determinar nivel de alerta
if dias_hasta_critico and dias_hasta_critico <= 7:
    # ALERTA CRÍTICA
    print("  ╔════════════════════════════════════════════════════════════════════════════╗")
    print("  ║                                                                            ║")
    print("  ║                        🔴 ALERTA CRÍTICA 🔴                                ║")
    print("  ║                                                                            ║")
    print("  ╚════════════════════════════════════════════════════════════════════════════╝")
    print()
    print(f"  ⚠️  NIVEL CRÍTICO SERÁ ALCANZADO EN {int(dias_hasta_critico)} DÍAS")
    print(f"  📅 Fecha estimada: {fecha_critico.strftime('%d/%m/%Y')}")
    print(f"  📉 Stock proyectado ese día: {UMBRAL_MINIMO:,.0f} bins (umbral crítico)")
    print()
    print("  🔔 ACCIONES REQUERIDAS INMEDIATAS:")
    print("     • Contactar a todos los productores con bins en campo")
    print("     • Priorizar devoluciones urgentes")
    print("     • Pausar nuevas entregas de bins vacíos")
    print("     • Reunión de emergencia con equipo operativo")
    print("     • Activar plan de contingencia")

elif dias_hasta_amarillo and dias_hasta_amarillo <= 7:
    # ADVERTENCIA
    print("  ╔════════════════════════════════════════════════════════════════════════════╗")
    print("  ║                                                                            ║")
    print("  ║                        🟡 ADVERTENCIA 🟡                                   ║")
    print("  ║                                                                            ║")
    print("  ╚════════════════════════════════════════════════════════════════════════════╝")
    print()
    print(f"  ⚠️  NIVEL DE ADVERTENCIA SERÁ ALCANZADO EN {int(dias_hasta_amarillo)} DÍAS")
    print(f"  📅 Fecha estimada: {fecha_amarillo.strftime('%d/%m/%Y')}")
    print(f"  📉 Stock proyectado ese día: {UMBRAL_AMARILLO:,.0f} bins (umbral amarillo)")
    print()
    print("  🔔 ACCIONES RECOMENDADAS:")
    print("     • Monitorear stock diariamente")
    print("     • Revisar productores con mayor deuda de bins")
    print("     • Planificar operativo de recuperación preventiva")
    print("     • Preparar comunicaciones para productores")

elif dias_hasta_critico and dias_hasta_critico <= 14:
    # PRECAUCIÓN (más de 7 días pero menos de 14)
    print("  ╔════════════════════════════════════════════════════════════════════════════╗")
    print("  ║                                                                            ║")
    print("  ║                        🟠 PRECAUCIÓN 🟠                                    ║")
    print("  ║                                                                            ║")
    print("  ╚════════════════════════════════════════════════════════════════════════════╝")
    print()
    print(f"  ⚠️  NIVEL CRÍTICO PROYECTADO EN {int(dias_hasta_critico)} DÍAS")
    print(f"  📅 Fecha estimada: {fecha_critico.strftime('%d/%m/%Y')}")
    print()
    print("  🔔 ACCIONES SUGERIDAS:")
    print("     • Mantener vigilancia sobre tendencia")
    print("     • Revisar proyecciones semanalmente")
    print("     • Identificar productores con bins ociosos")

else:
    # SIN RIESGO
    print("  ╔════════════════════════════════════════════════════════════════════════════╗")
    print("  ║                                                                            ║")
    print("  ║                        🟢 SITUACIÓN NORMAL 🟢                              ║")
    print("  ║                                                                            ║")
    print("  ╚════════════════════════════════════════════════════════════════════════════╝")
    print()
    print("  ✅ Stock proyectado se mantiene en niveles seguros")
    print("  ✅ Sin alertas para los próximos 7 días")
    print()
    print("  📋 RECOMENDACIONES:")
    print("     • Continuar con operación normal")
    print("     • Mantener monitoreo rutinario")

print()
print("=" * 80)
print()

# ============================================================================
# RESUMEN EJECUTIVO (para copiar/pegar en reportes)
# ============================================================================

print("📄 RESUMEN EJECUTIVO")
print("-" * 80)

if dias_hasta_critico and dias_hasta_critico <= 7:
    print(f"Stock actual: {stock_actual:,.0f} bins | Tendencia: {pendiente:,.1f} bins/día (BAJANDO)")
    print(f"⚠️ CRÍTICO: Nivel crítico en {int(dias_hasta_critico)} días ({fecha_critico.strftime('%d/%m/%Y')})")
    print("Acción: URGENTE - Activar plan de contingencia inmediato")
elif dias_hasta_amarillo and dias_hasta_amarillo <= 7:
    print(f"Stock actual: {stock_actual:,.0f} bins | Tendencia: {pendiente:,.1f} bins/día")
    print(f"⚠️ ADVERTENCIA: Nivel amarillo en {int(dias_hasta_amarillo)} días ({fecha_amarillo.strftime('%d/%m/%Y')})")
    print("Acción: Monitorear de cerca y preparar medidas preventivas")
else:
    print(f"Stock actual: {stock_actual:,.0f} bins | Tendencia: {pendiente:,.1f} bins/día")
    print("✅ Sin alertas - Situación bajo control")

print()
print("=" * 80)
print("✅ Análisis de alerta completado")
print()


In [ ]:
# ============================================================================
# CELDA 7: GRÁFICO: PLANTA VS CAMPOS (DOS LÍNEAS)
# ============================================================================

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

print("Generando gráfico de planta vs campos...")

# Crear figura
fig, ax = plt.subplots(figsize=(18, 9))

# ============================================================================
# LÍNEAS: Planta (verde) y Campos (roja)
# ============================================================================

# Línea verde: bins disponibles en planta
ax.plot(balance_diario['FECHA_DIA'],
        balance_diario['BINS_EN_PLANTA'],
        color='#2ECC71', linewidth=4, label='Bins en PLANTA (disponibles)', zorder=3)

# Línea roja: bins comprometidos en campos
ax.plot(balance_diario['FECHA_DIA'],
        balance_diario['BINS_EN_CAMPOS'],
        color='#E74C3C', linewidth=4, label='Bins en CAMPOS (comprometidos)', zorder=3)

# Área sombreada bajo la línea de planta
ax.fill_between(balance_diario['FECHA_DIA'],
                0, balance_diario['BINS_EN_PLANTA'],
                color='#2ECC71', alpha=0.2)

# ============================================================================
# LÍNEAS DE REFERENCIA
# ============================================================================

# Línea negra: total del sistema
ax.axhline(y=TOTAL_BINS, color='black', linestyle='-', linewidth=2,
           alpha=0.5, label=f'Total sistema: {TOTAL_BINS:,} bins', zorder=5)

# Línea naranja: umbral de advertencia
ax.axhline(y=UMBRAL_AMARILLO, color='orange', linestyle='--', linewidth=2,
           label=f'Advertencia: {UMBRAL_AMARILLO:,} bins', alpha=0.8, zorder=5)

# Línea roja: umbral crítico
ax.axhline(y=UMBRAL_MINIMO, color='darkred', linestyle='--', linewidth=3,
           label=f'CRÍTICO: {UMBRAL_MINIMO:,} bins', alpha=0.9, zorder=5)

# ============================================================================
# MARCADORES: Días críticos
# ============================================================================

dias_criticos = balance_diario[balance_diario['NIVEL_ALERTA'] == 'CRITICO']
if len(dias_criticos) > 0:
    ax.scatter(dias_criticos['FECHA_DIA'], dias_criticos['BINS_EN_PLANTA'],
               color='darkred', s=150, marker='X', linewidths=3,
               edgecolors='white', zorder=10, label='Días CRÍTICOS')

# ============================================================================
# FORMATO DEL GRÁFICO
# ============================================================================

# Títulos
ax.set_title('Stock de Bins: Planta vs Campos\nJugos S.A.',
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Fecha', fontsize=14, fontweight='bold')
ax.set_ylabel('Cantidad de Bins', fontsize=14, fontweight='bold')

# Eje Y: formato con separadores de miles
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x):,}'))
ax.set_ylim(0, TOTAL_BINS + 2000)

# Eje X: fechas cada 2 semanas
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m/%Y'))
ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=2))
plt.xticks(rotation=45, ha='right')

# Grid y leyenda
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
ax.legend(loc='upper right', fontsize=12, framealpha=0.95, shadow=True)

plt.tight_layout()
plt.show()

print("✅ Gráfico generado correctamente")

In [ ]:
# ============================================================================
# CELDA 8: RESUMEN EJECUTIVO
# ============================================================================

print("="*80)
print("📊 JUGOS S.A. - RESUMEN EJECUTIVO")
print("="*80)
print(f"Stock actual: {balance_diario['BINS_EN_PLANTA'].iloc[-1]:,} bins")
print(f"En campos: {balance_diario['BINS_EN_CAMPOS'].iloc[-1]:,} bins")
print(f"Estado: {balance_diario['NIVEL_ALERTA'].iloc[-1]}")
if 'alerta_texto' in locals() and alerta_texto:
    print(f"\n⚠️ {alerta_texto}")
print("="*80)